In [1]:
import os
import json

import torch
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd

import torch_utils
from torch import distributions

import gymnasium as gym
import gymnasium_robotics as gym_robo
import models
import cnn_models
import rl_agents
import rl_callbacks
import helper
import gym_helper
import wandb_support
import wandb
import gym_helper

# from mpi4py import MPI

In [ ]:
import mujoco

In [ ]:
mujoco.MjModel

In [ ]:
gym_robo.__version__

In [ ]:
def get_default_device():
    """Returns the default device for computations, GPU if available, otherwise CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()
print(f"Using device: {device}")

# TEST

In [ ]:
gym_robo.register_robotics_envs()

In [ ]:
gym.envs.registration.registry

In [ ]:
wandb.login(key='758ac5ba01e12a3df504d2db2fec8ba4f391f7e6')

In [ ]:
env = gym.make('FetchPush-v2', max_episode_steps=100, render_mode='rgb_array')
env = gym.wrappers.RecordVideo(env, 'test/', episode_trigger=lambda i: i%1==0)

episodes = 10


for episode in range(episodes):
    done = False
    obs, _ = env.reset()
    while not done:
        obs, r, term, trunc, dict = env.step(env.action_space.sample())
        if term or trunc:
            done = True
env.close()

In [ ]:
env = gym.make("FetchReach-v2")
env.reset()
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())

# The following always has to hold:
assert reward == env.compute_reward(obs["achieved_goal"], obs["desired_goal"], info)
assert truncated == env.compute_truncated(obs["achieved_goal"], obs["desired_goal"], info)
assert terminated == env.compute_terminated(obs["achieved_goal"], obs["desired_goal"], info)

In [ ]:
env.compute_reward()

In [ ]:
env = gym.make('FetchPush-v2', render_mode='rgb_array')

In [ ]:
if hasattr(env, "distance_threshold"):
    print('true')
else:
    print('false')

In [ ]:
if env.get_wrapper_attr("distance_threshold"):
    print('true')

In [ ]:
print(dir(env))


# DDPG

In [ ]:
env = gym.make('LunarLanderContinuous-v2')

In [ ]:
device='cuda'

In [ ]:
# # build actor

# dense_layers = [
#     (
#         400,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     ),
#     (
#         300,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     )
# ]

# actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, optimizer='Adam',
#                           optimizer_params={'weight_decay':0.01}, learning_rate=0.001, normalize_layers=True)

In [ ]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        300,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, output_layer_kernel=output_kernel,
                          optimizer='Adam', optimizer_params={'weight_decay':0.0}, learning_rate=0.001,
                          normalize_layers=False, device=device)

In [ ]:
actor

In [ ]:
# # build critic

# state_layers = [
#     (
#         400,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     )
# ]

# merged_layers = [
#     (
#         300,
#         "relu",
#         {
#             "variance scaling": {
#                 "scale": 1.0,
#                 "mode": "fan_in",
#                 "distribution": "uniform",
#             }
#         },
#     )
# ]


# critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
#                             optimizer='Adam', optimizer_params={'weight_decay':0.01}, learning_rate=0.002, normalize_layers=True)

In [ ]:
# build critic
# build actor

state_layers = []

merged_layers = [
    (
        400,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        300,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
                            output_layer_kernel=output_kernel, optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},learning_rate=0.001, normalize_layers=False,
                            device=device)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 100000, device=device)
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.1, device=device)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.99,
                            tau=0.005,
                            replay_buffer=replay_buffer,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('LunarLander-v2-continuous')],
                            device=device)

In [ ]:
ddpg_agent.critic_model

In [ ]:
ddpg_agent.target_critic_model

In [ ]:
ddpg_agent.train(500)

In [ ]:
ddpg_agent.test(10, True, 1)

# Actor Critic

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
dense_layers = [
    (128, 'relu', "kaiming normal"),
    (256, 'relu', "kaiming normal"),
    ]



In [ ]:
policy_model = models.PolicyModel(env=env, dense_layers=dense_layers, optimizer='Adam', learning_rate=0.001,)

In [ ]:
for param in policy_model.parameters():
    print(param)

In [ ]:
value_model = models.ValueModel(env, dense_layers=dense_layers, optimizer='Adam', learning_rate=0.001)

In [ ]:
value_model

In [ ]:
for params in value_model.parameters():
    print(params)

In [ ]:
actor_critic = rl_agents.ActorCritic(env,
                                     policy_model,
                                     value_model,
                                     discount=0.99,
                                     policy_trace_decay=0.5,
                                     value_trace_decay=0.5,
                                     callbacks=[rl_callbacks.WandbCallback('CartPole-v1-Actor-Critic')])

In [ ]:
actor_critic.train(200)

In [ ]:
actor_critic.test(10, True, 1)

# REINFORCE

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
dense_layers = [
    (128, 'relu', {
                    "kaiming normal": {
                        "a":1.0,
                        "mode":'fan_in'
                    }
                },
    ),
    # (256, 'relu', {
    #                 "kaiming_normal": {
    #                     "a":0.0,
    #                     "mode":'fan_in'
    #                 }
    #             },
    # )
    ]

In [ ]:
dense_layers = [(128, 'relu', "kaiming normal")]

In [ ]:
value_model = models.ValueModel(env, dense_layers, 'Adam', 0.001)

In [ ]:
for param in value_model.parameters():
    print(param)

In [ ]:
policy_model = models.PolicyModel(env, dense_layers, 'Adam', 0.001)

In [ ]:
for param in policy_model.parameters():
    print(param)

In [ ]:
reinforce = rl_agents.Reinforce(env, policy_model, value_model, 0.99, [rl_callbacks.WandbCallback('CartPole-v0_REINFORCE', chkpt_freq=100)])

In [ ]:
reinforce.train(200, True, 50)

In [ ]:
reinforce.test(10, True, 1)

# DDPG w/CNN

In [ ]:
env = gym.make('CarRacing-v2')

In [ ]:
cnn_layers = [
    # {
    #     "batchnorm":
    #     {
    #         "num_features":3
    #     }
    # },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 7,
            "stride": 3,
            "padding": 'valid',
            "bias": False
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 5,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 3,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
]

In [ ]:
cnn = cnn_models.CNN(cnn_layers, env)

In [ ]:
cnn

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env, cnn_model=cnn, dense_layers=dense_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.0001, normalize=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=cnn, state_layers=state_layers, merged_layers=merged_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.0001, normalize=False)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape=(1,))
noise = helper.OUNoise(shape=env.action_space.shape, mean=0.0, theta=0.15, sigma=0.01, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(
    env,
    actor,
    critic,
    discount=0.98,
    tau=0.05,
    action_epsilon=0.2,
    replay_buffer=replay_buffer,
    batch_size=128,
    noise=noise,
    callbacks=[rl_callbacks.WandbCallback("CarRacing-v2")]
)

In [ ]:
ddpg_agent.train(1000, True, 10)

In [ ]:
wandb.finish()

In [ ]:
wandb.login()

# HER

In [ ]:
env = gym.make("Reacher-v4")

In [ ]:
_,_ = env.reset()

In [ ]:
achieved_goal = gym_helper.reacher_achieved_goal(env)
action = env.action_space.sample()
env.step(action)
print(f'observation: {env.get_wrapper_attr("_get_obs")()}')
print(f'distance to goal: {env.get_wrapper_attr("_get_obs")()[8::]}')
print(f'fingertip: {env.get_wrapper_attr("get_body_com")("fingertip")}')
print(f'target: {env.get_wrapper_attr("get_body_com")("target")}')

In [ ]:
next_achieved_goal = env.get_wrapper_attr("_get_obs")()[8::]
desired_goal = [0.0, 0.0, 0.0]

In [ ]:
reward_func(env, action, achieved_goal, next_achieved_goal, desired_goal, 0.05)

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env,
                          cnn_model=None,
                          dense_layers=dense_layers,
                          goal_shape=(3,),
                          optimizer="Adam",
                          optimizer_params={'weight_decay':0.0},
                          learning_rate=0.0001, normalize=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env,
                            cnn_model=None,
                            state_layers=state_layers,
                            merged_layers=merged_layers,
                            goal_shape=(3,),
                            optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},
                            learning_rate=0.0001,
                            normalize=False)

In [ ]:
goal_shape = desired_goal_func(env).shape
replay_buffer = helper.ReplayBuffer(env, 100000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape,
#                        mean=0.0,
#                        theta=0.05,
#                        sigma=0.15,
#                        dt=1.0, device='cuda')

noise=helper.NormalNoise(shape=env.action_space.shape,
                         mean = 0.0,
                         stddev=0.05,
                         )

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Reacher-v4')])

In [ ]:
her = rl_agents.HER(ddpg_agent,
                    strategy='future',
                    num_goals=4,
                    tolerance=0.001,
                    desired_goal=desired_goal_func,
                    achieved_goal=achieved_goal_func,
                    reward_fn=reward_func)

In [ ]:
her.train(10, 50, 16, 40, True, 1000)

In [ ]:
wandb.finish()

In [ ]:
her.test(10, True, 1)

In [ ]:
her.save()

In [ ]:
her.agent.goal_normalizer.running_std

In [ ]:
loaded_her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/assets/models/her")

In [ ]:
loaded_her.agent.replay_buffer.sample(10)

In [ ]:
loaded_her.agent.state_normalizer.running_cnt

In [ ]:
loaded_her.get_config()

In [ ]:
loaded_her.test(10, True, 1)

In [ ]:
10e4

# HER w/CNN

In [ ]:
env = gym.make('CarRacing-v2')

In [ ]:
_,_ = env.reset()

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
desired_goal(env).shape

In [ ]:
cnn_layers = [
    # {
    #     "batchnorm":
    #     {
    #         "num_features":3
    #     }
    # },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 7,
            "stride": 3,
            "padding": 'valid',
            "bias": False
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 5,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 3,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
]

cnn = cnn_models.CNN(cnn_layers, env)

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env,
                          cnn_model=cnn,
                          dense_layers=dense_layers,
                          goal_shape=(1,),
                          optimizer="Adam",
                          optimizer_params={'weight_decay':0.0},
                          learning_rate=0.001, normalize=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env,
                            cnn_model=cnn,
                            state_layers=state_layers,
                            merged_layers=merged_layers,
                            goal_shape=(1,),
                            optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},
                            learning_rate=0.001,
                            normalize=False)

In [ ]:
critic

In [ ]:
goal_shape = desired_goal_func(env).shape
replay_buffer = helper.ReplayBuffer(env, 100000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape,
#                        mean=0.0,
#                        theta=0.05,
#                        sigma=0.15,
#                        dt=1.0, device='cuda')

noise=helper.NormalNoise(shape=env.action_space.shape,
                         mean = 0.0,
                         stddev=0.05,
                         )

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('CarRacing-v2')])

In [ ]:
ddpg_agent.actor_model

In [ ]:
her = rl_agents.HER(ddpg_agent,
                    strategy='future',
                    num_goals=4,
                    tolerance=1,
                    desired_goal=desired_goal_func,
                    achieved_goal=achieved_goal_func,
                    reward_fn=reward_func)

In [ ]:
her.agent.actor_model

In [ ]:
her.train(num_epochs=20,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=20
        )

In [ ]:
her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/models/her")

In [ ]:
wandb.finish()

In [ ]:
# reset environment
state, _ = her.agent.env.reset()
# instantiate empty lists to store current episode trajectory
states, actions, next_states, dones, state_achieved_goals, \
next_state_achieved_goals, desired_goals = [], [], [], [], [], [], []
# set desired goal
desired_goal = her.desired_goal_func(her.agent.env)
# set achieved goal
state_achieved_goal = her.achieved_goal_func(her.agent.env)
# add initial state and goals to local normalizer stats
her.state_normalizer.update_local_stats(state)
her.goal_normalizer.update_local_stats(desired_goal)
her.goal_normalizer.update_local_stats(state_achieved_goal)
# set done flag
done = False
# reset episode reward to 0
episode_reward = 0
# reset steps counter for the episode
episode_steps = 0

while not done:
    # get normalized values for state and desired goal
    state_norm = her.state_normalizer.normalize(state)
    desired_goal_norm = her.goal_normalizer.normalize(desired_goal)
    # get action
    action = her.agent.get_action(state_norm, desired_goal_norm, grad=False)
    # take action
    next_state, reward, term, trunc, _ = her.agent.env.step(action)
    # get next state achieved goal
    next_state_achieved_goal = her.achieved_goal_func(her.agent.env)
    # add next state and next state achieved goal to normalizers
    her.state_normalizer.update_local_stats(next_state)
    her.goal_normalizer.update_local_stats(next_state_achieved_goal)
    # store trajectory in replay buffer (non normalized!)
    her.agent.replay_buffer.add(state, action, reward, next_state, done,\
                                    state_achieved_goal, next_state_achieved_goal, desired_goal)
    
    # append step state, action, next state, and goals to respective lists
    states.append(state)
    actions.append(action)
    next_states.append(next_state)
    dones.append(done)
    state_achieved_goals.append(state_achieved_goal)
    next_state_achieved_goals.append(next_state_achieved_goal)
    desired_goals.append(desired_goal)

    # add to episode reward and increment steps counter
    episode_reward += reward
    episode_steps += 1
    # update state and state achieved goal
    state = next_state
    state_achieved_goal = next_state_achieved_goal
    # update done flag
    if term or trunc:
        done = True

In [ ]:
# package episode states, actions, next states, and goals into trajectory tuple
trajectory = (states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)

In [ ]:
states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals = trajectory

In [ ]:
for idx, (s, a, ns, d, sag, nsag, dg) in enumerate(zip(states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)):
    print(f'a={a}, d={d}, sag={sag}, nsag={nsag}, dg={dg}')

In [ ]:
strategy = "future"
num_goals = 4

# loop over each step in the trajectory to set new achieved goals, calculate new reward, and save to replay buffer
for idx, (state, action, next_state, done, state_achieved_goal, next_state_achieved_goal, desired_goal) in enumerate(zip(states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)):

    if strategy == "final":
        new_desired_goal = next_state_achieved_goals[-1]
        new_reward = her.reward_fn(state_achieved_goal, next_state_achieved_goal, new_desired_goal)
        print(f'transition: action={action}, reward={new_reward}, done={done}, state_achieved_goal={state_achieved_goal}, next_state_achieved_goal={next_state_achieved_goal}, desired_goal={new_desired_goal}')
        her.agent.replay_buffer.add(state, action, new_reward, next_state, done, state_achieved_goal, next_state_achieved_goal, new_desired_goal)

    if strategy == 'future':
        for i in range(num_goals):
            if idx + i + 1 >= len(states):
                break
            goal_idx = np.random.randint(idx + 1, len(states))
            new_desired_goal = next_state_achieved_goals[goal_idx]
            new_reward = her.reward_fn(state_achieved_goal, next_state_achieved_goal, new_desired_goal)
            print(f'transition: action={action}, reward={new_reward}, done={done}, state_achieved_goal={state_achieved_goal}, next_state_achieved_goal={next_state_achieved_goal}, desired_goal={new_desired_goal}')
            her.agent.replay_buffer.add(state, action, new_reward, next_state, done, state_achieved_goal, next_state_achieved_goal, new_desired_goal)
    

    


In [ ]:
s, a, r, ns, d, sag, nsag, dg = her.agent.replay_buffer.sample(100)

In [ ]:
for i in range(100):
    print(f'{i}: a={a[i]}, r={r[i]}, d={d[i]}, sag={sag[i]}, nsag={nsag[i]}, dg={dg[i]} ')

# HER Pendulum

In [ ]:
env = gym.make('Pendulum-v1')

In [ ]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        300,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, optimizer='Adam',
                          optimizer_params={'weight_decay':0.01}, learning_rate=0.001, normalize=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.001, normalize=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 100000, (3,))
noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.99,
                            tau=0.005,
                            replay_buffer=replay_buffer,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Pendulum-v1')])

In [ ]:
def desired_goal_func(env):
    return np.array([0.0, 0.0, 0.0])

def achieved_goal_func(env):
    return env.get_wrapper_attr('_get_obs')()

def reward_func(env):
    pass

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='none',
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=10.0
)

In [ ]:
her.agent.critic_model

In [ ]:
her.agent.target_critic_model

In [ ]:
her.train(1,1,100,1)

In [ ]:
wandb.finish()

In [ ]:
state = env.observation_space.sample()
state

In [ ]:
her.agent.state_normalizer.normalize(state)

In [ ]:
goal = her.desired_goal_func(her.agent.env)
goal

In [ ]:
her.agent.goal_normalizer.normalize(goal)

In [ ]:
def remove_renders(folder_path):
    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file has a .mp4 or .meta.json extension
        if filename.endswith(".mp4") or filename.endswith(".meta.json"):
            # Construct the full file path
            file_path = os.path.join(folder_path, filename)
            # Remove the file
            os.remove(file_path)

In [ ]:
remove_renders("/workspaces/RL_Agents/pytorch/src/app/assets/models/ddpg/renders/training")

# HER Fetch-Reach (Robotics)

In [ ]:
env = gym.make("FetchReach-v2", max_episode_steps=50)

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
achieved_goal_func(env)

In [ ]:
env.get_wrapper_attr("_get_obs")()

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
goal_shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchReach-v2")])

In [ ]:
ddpg_agent.critic_model

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='future',
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0
)

In [ ]:
her.train(num_epochs=50,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=1000)

In [ ]:
states, action, rewards, next_states, dones, achieved_goals, next_achieved_goals, desired_goals = her.agent.replay_buffer.sample(2)

In [ ]:
desired_goals

In [ ]:
her.agent.env.get_wrapper_attr("distance_threshold")

In [ ]:
# get success
her.agent.env.get_wrapper_attr("_is_success")(achieved_goal_func(her.agent.env), desired_goal_func(her.agent.env))

In [ ]:
her.agent.env.get_wrapper_attr("goal_distance")(next_state_achieved_goal, desired_goal, None)

In [ ]:
pusher_her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/assets/models/her")

In [ ]:
pusher_her.agent.env.reset()

In [ ]:
pusher_her.get_config()

In [ ]:
wandb.finish()

In [ ]:
np.linalg.norm(pusher_her.agent.env.get_wrapper_attr("get_body_com")("goal") - pusher_her.agent.env.get_wrapper_attr("get_body_com")("object"))

In [ ]:
pusher_her.agent.replay_buffer.get_config()

In [ ]:

pusher_her.agent.replay_buffer.desired_goals

In [ ]:
## TEST ENV
env = gym.make("Pusher-v5", render_mode="rgb_array")

In [ ]:
env = gym.wrappers.RecordVideo(
                    env,
                    "/renders/training",
                    episode_trigger=lambda x: True,
                )


In [ ]:
state, _ = env.reset()

for i in range(1000):
# take action
    next_state, reward, term, trunc, _ = env.step(env.action_space.sample())
env.close()

# HER Fetch Push (Robitics)

In [ ]:
env = gym.make('FetchPush-v2')

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.3,
                            replay_buffer=replay_buffer,
                            batch_size=128,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchPush-v2")],
                            save_dir="fetch_push/models/ddpg/"
                            )

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='final',
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0,
    save_dir="fetch_push/models/her/"
)

In [ ]:
her.train(num_epochs=50,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=1000)

# TESTING MULTITHREADING

In [ ]:
env = gym.make('FetchPush-v2')

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.3,
                            replay_buffer=replay_buffer,
                            batch_size=128,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchPush-v2")],
                            save_dir="fetch_push/models/ddpg/"
                            )

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='final',
    num_workers=4,
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0,
    save_dir="fetch_push/models/her/"
)

In [ ]:
her.train()

# TESTING

In [ ]:
# load config
config_path = "/workspaces/RL_Agents/pytorch/src/app/HER_Test/her/config.json"
with open(config_path, 'r') as file:
    config = json.load(file)

In [ ]:
config

In [ ]:
agent = rl_agents.HER.load(config)

In [ ]:
for callback in agent.agent.callbacks:
    print(callback._sweep)

# Co Occurence

In [ ]:
import subprocess

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'assets/wandb_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    wandb_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(wandb_config)

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'assets/sweep_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    sweep_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(sweep_config)

In [ ]:
# Save the updated configuration to a train config file
os.makedirs('sweep', exist_ok=True)
train_config_path = os.path.join(os.getcwd(), 'sweep/train_config.json')
with open(train_config_path, 'w') as f:
    json.dump(sweep_config, f)

# Save and Set the sweep config path
sweep_config_path = os.path.join(os.getcwd(), 'sweep/sweep_config.json')
with open(sweep_config_path, 'w') as f:
    json.dump(wandb_config, f)

In [ ]:
command = ['python', 'sweep.py']

# Set the environment variable
os.environ['WANDB_DISABLE_SERVICE'] = 'true'

subprocess.Popen(command)

In [ ]:
# Set the environment variable
os.environ['WANDB_DISABLE_SERVICE'] = 'true'

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'sweep/sweep_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    sweep_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(sweep_config)

In [ ]:
# Define the path to your JSON configuration file
config_file_path = 'sweep/train_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    train_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(train_config)

In [ ]:
from dash_callbacks import run_agent

sweep_id = wandb.sweep(sweep=sweep_config, project=sweep_config["project"])
# loop over num wandb agents
num_agents = 2
for agent in range(num_agents):
    
)

In [ ]:
sweep_config

In [ ]:
env = gym.make("FetchReach-v2")

In [ ]:
type(env)

In [ ]:
env_spec = env.spec.to_json()

In [ ]:
env_spec

In [ ]:
type(env_spec)

In [ ]:
env = gym.make(gym.envs.registration.EnvSpec.from_json(env_spec))

In [ ]:
env.spec

In [ ]:
def load_env_spec(env_spec_dict):
    # Create a new EnvSpec instance using the dictionary
    env_spec = gym.envs.registration.EnvSpec(**env_spec_dict)
    return env_spec

In [ ]:
load_env_spec(env_spec)

In [ ]:
config_path = 'sweep/agent_config_58.json'

with open(config_path, 'r') as file:
    agent_config = json.load(file)

her = rl_agents.HER.load(agent_config)

In [ ]:
her.get_config()

# TD3

In [72]:
env = gym.make('LunarLanderContinuous-v2')
# env = gym.make("BipedalWalker-v3")
# env = gym.make("Pendulum-v1")

In [73]:
device='cuda'

In [74]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        300,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, output_layer_kernel=output_kernel,
                          optimizer='Adam', optimizer_params={'weight_decay':0.0}, learning_rate=0.001,
                          normalize_layers=False, device=device)

In [75]:
actor

ActorModel(
  (dense_layers): ModuleDict(
    (actor_dense_0): Linear(in_features=8, out_features=400, bias=True)
    (actor_activation_0): ReLU()
    (actor_dense_1): Linear(in_features=400, out_features=300, bias=True)
    (actor_activation_1): ReLU()
  )
  (output_layer): ModuleDict(
    (actor_dense_output): Linear(in_features=300, out_features=2, bias=True)
  )
  (output_activation): ModuleDict(
    (actor_output_activation): Tanh()
  )
)

In [76]:
for param in actor.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0115, -0.0610,  0.2034,  ...,  0.0929,  0.1644, -0.2142],
        [-0.2293,  0.3289,  0.2403,  ..., -0.2239, -0.0866, -0.0815],
        [ 0.3203,  0.1492,  0.2891,  ...,  0.1638,  0.0425,  0.2894],
        ...,
        [-0.3322,  0.0452, -0.1554,  ..., -0.1615,  0.2839, -0.3025],
        [-0.0889, -0.1612, -0.3380,  ..., -0.1885,  0.0626, -0.0231],
        [-0.2954,  0.2041,  0.2984,  ...,  0.0191,  0.1847, -0.2045]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-2.3757e-01, -2.4925e-01, -2.1249e-01, -4.1449e-02,  5.4232e-03,
         1.5996e-01, -9.0564e-02, -1.3755e-01,  1.0248e-01,  4.1701e-02,
        -2.5326e-01,  1.0143e-01, -8.3967e-02, -2.0217e-02, -2.7982e-01,
         2.5943e-01, -1.0887e-02, -6.0356e-03,  1.3281e-02,  1.9203e-01,
        -5.9374e-02, -4.5049e-02,  9.6441e-02, -1.4843e-02,  2.2055e-01,
         1.9549e-01,  2.2418e-01,  1.7675e-01,  1.6141e-01, -2.2021e-01,
         2.9091e-01, -3.8986e-02, -7.457

In [77]:
# build critic
# build actor

state_layers = []

merged_layers = [
    (
        400,
        "relu",
        {
            "default": {
                
            }
        },
    ),
    (
        300,
        "relu",
        {
            "default": {
                
            }
        },
    )
]

# output_kernel = {"uniform":{"a":-0.003, "b":0.003}}
output_kernel = {"default":{}}

critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
                            output_layer_kernel=output_kernel, optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},learning_rate=0.001, normalize_layers=False,
                            device=device)

In [78]:
critic

CriticModel(
  (state_layers): ModuleDict()
  (merged_layers): ModuleDict(
    (critic_merged_dense_0): Linear(in_features=10, out_features=400, bias=True)
    (critic_merged_activation_0): ReLU()
    (critic_merged_dense_1): Linear(in_features=400, out_features=300, bias=True)
    (critic_merged_activation_1): ReLU()
  )
  (output_layer): ModuleDict(
    (critic_dense_output): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [79]:
for params in critic.parameters():
    print(params)

Parameter containing:
tensor([[-0.0712, -0.1902,  0.2527,  ..., -0.2434, -0.1654, -0.1265],
        [-0.2965, -0.1451, -0.0214,  ...,  0.2057,  0.1145, -0.1678],
        [-0.2875,  0.2559, -0.2351,  ..., -0.1548,  0.3007, -0.2464],
        ...,
        [-0.0359,  0.0264,  0.2237,  ...,  0.1505,  0.1715, -0.2849],
        [-0.2568,  0.1468,  0.1647,  ...,  0.0205,  0.1647, -0.0707],
        [-0.0400, -0.1519,  0.1945,  ...,  0.2025, -0.2539,  0.1306]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2519,  0.1202, -0.3138, -0.1956,  0.0276,  0.2153, -0.0290, -0.0130,
        -0.1154,  0.0362,  0.2828, -0.0678, -0.0658,  0.0395, -0.0302,  0.2040,
        -0.0854,  0.2761,  0.0887, -0.1521, -0.0403,  0.1347, -0.1413, -0.2800,
        -0.1546,  0.1733,  0.1529, -0.0211, -0.2887,  0.0799,  0.2062,  0.2759,
        -0.0553, -0.2730,  0.0537, -0.0979,  0.2418, -0.2705, -0.1285, -0.2905,
        -0.2717, -0.2390,  0.2196,  0.1011, -0.0285,  0.2001, -0.0085,  0.0662

In [80]:
replay_buffer = helper.ReplayBuffer(env, 1000000, device=device)
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.1, device=device)

In [81]:
td3 = rl_agents.TD3(
    env=env,
    actor_model=actor,
    critic_model=critic,
    replay_buffer=replay_buffer,
    batch_size=100,
    noise=noise,
    actor_update_delay=2,
    callbacks=[rl_callbacks.WandbCallback("LunarLander-v2-continuous")],
    use_mpi=False,
    device=device
)

In [82]:
for params in td3.critic_model_a.parameters():
    print(params)

Parameter containing:
tensor([[-0.0712, -0.1902,  0.2527,  ..., -0.2434, -0.1654, -0.1265],
        [-0.2965, -0.1451, -0.0214,  ...,  0.2057,  0.1145, -0.1678],
        [-0.2875,  0.2559, -0.2351,  ..., -0.1548,  0.3007, -0.2464],
        ...,
        [-0.0359,  0.0264,  0.2237,  ...,  0.1505,  0.1715, -0.2849],
        [-0.2568,  0.1468,  0.1647,  ...,  0.0205,  0.1647, -0.0707],
        [-0.0400, -0.1519,  0.1945,  ...,  0.2025, -0.2539,  0.1306]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2519,  0.1202, -0.3138, -0.1956,  0.0276,  0.2153, -0.0290, -0.0130,
        -0.1154,  0.0362,  0.2828, -0.0678, -0.0658,  0.0395, -0.0302,  0.2040,
        -0.0854,  0.2761,  0.0887, -0.1521, -0.0403,  0.1347, -0.1413, -0.2800,
        -0.1546,  0.1733,  0.1529, -0.0211, -0.2887,  0.0799,  0.2062,  0.2759,
        -0.0553, -0.2730,  0.0537, -0.0979,  0.2418, -0.2705, -0.1285, -0.2905,
        -0.2717, -0.2390,  0.2196,  0.1011, -0.0285,  0.2001, -0.0085,  0.0662

In [83]:
for params in td3.critic_model_b.parameters():
    print(params)

Parameter containing:
tensor([[-0.1547, -0.2648, -0.1211,  ..., -0.1472,  0.2410,  0.1181],
        [-0.2133,  0.2835, -0.1802,  ...,  0.2814, -0.0767, -0.1798],
        [-0.1346, -0.1935,  0.0492,  ..., -0.2078,  0.1841, -0.2975],
        ...,
        [-0.2765,  0.3075,  0.1296,  ..., -0.0707, -0.3066, -0.1308],
        [-0.1368, -0.1513,  0.2212,  ...,  0.2524, -0.2108,  0.1220],
        [ 0.1606, -0.1621,  0.2802,  ..., -0.0578,  0.0556,  0.0196]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0157, -0.2202,  0.3058, -0.0915,  0.1351,  0.1984, -0.2734, -0.2161,
        -0.0370,  0.2529, -0.2281, -0.2630, -0.0269, -0.0878,  0.2586,  0.1481,
        -0.0790,  0.2680,  0.0130,  0.1794, -0.0803,  0.1133,  0.2156, -0.0124,
         0.1778, -0.2895,  0.2227, -0.1976, -0.1114, -0.0266,  0.1798, -0.1544,
        -0.2650,  0.1687, -0.1883, -0.1235, -0.0769,  0.0292,  0.1421, -0.0554,
        -0.1995,  0.1551, -0.1208,  0.2292, -0.1994, -0.1359,  0.2192, -0.2423

In [84]:
td3.get_config()

{'agent_type': 'TD3',
 'env': '{"id": "LunarLanderContinuous-v2", "entry_point": "gymnasium.envs.box2d.lunar_lander:LunarLander", "reward_threshold": 200, "nondeterministic": false, "max_episode_steps": 1000, "order_enforce": true, "disable_env_checker": false, "kwargs": {"continuous": true}, "additional_wrappers": [], "vector_entry_point": null}',
 'actor_model': {'env': 'LunarLanderContinuous-v2',
  'cnn_model': None,
  'num_layers': 4,
  'dense_layers': [(400, 'relu', {'default': {}}),
   (300, 'relu', {'default': {}})],
  'output_layer_kernel': {'default': {}},
  'goal_shape': None,
  'optimizer': 'Adam',
  'optimizer_params': {'weight_decay': 0.0},
  'learning_rate': 0.001,
  'normalize_layers': False,
  'clamp_output': None},
 'critic_model': {'env': 'LunarLanderContinuous-v2',
  'cnn_model': None,
  'num_layers': 4,
  'state_layers': [],
  'merged_layers': [(400, 'relu', {'default': {}}),
   (300, 'relu', {'default': {}})],
  'output_layer_kernel': {'default': {}},
  'goal_shape

In [85]:
td3.train(num_episodes=1500)

action_0,▄▄▅█▆▆▅▄▃█▄▇▅▄▇▆▅▅▅▆▇▆▆▆▅▅▆▅██▅▁▇▇▅▅▆▄▇▇
action_1,▄▄▅▁▄▅▆▃▄▃▄▃▃▄▅▄▇▄▂▅▅▅▆▅▆▅▆▆▅▃▅█▂▂▃▅▄▇▅▁
actor_loss,▂▂▁▂▁▁▁▁▂▂▂▁▃▂▃▂▂▁▂▃▁▂▂▁▂▁▂▄▄▂▂▇▃▅▃██▇▃▃
actor_predictions,█▁▃▃▄▅▃▂▄▆▅▆▄▄▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▄▄▄▄
avg_reward,▅▇███▆▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
best,███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
critic_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁
critic_predictions,▇▇█▇████▇▇▇█▆▇▆▇▇█▇▆█▇▇█▇█▇▅▅▇▇▂▆▄▆▁▁▂▆▆
episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode_reward,▆▇█▇█▁▄▅▆▆▄▄▆▅▄▄▅▄▃▁▃▆▇▅▃▆▆▅▄▄▆▆▅▄▅▅▅▄▅▃
episode_time,▁▃▃▃▃▃▃▅▃▄▅▄▅▃▅▆▄▇▅█▄▃▄▇▄▇▅▆▄▃▅▅▅▅▄▆▄▇▄▅


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide

wandb: Adding directory to artifact (./models/td3)... Done. 0.0s


episode 1, score -179.0056966120829, avg_score -179.0056966120829, episode_time 0.08s, avg_episode_time 0.08s, avg_step_time 0.000711s, avg_learn_time nans, avg_steps_per_episode 95.00
episode 2, score -117.69991516864908, avg_score -148.35280589036597, episode_time 0.76s, avg_episode_time 0.42s, avg_step_time 0.006604s, avg_learn_time 0.006006s, avg_steps_per_episode 101.00
episode 3, score -265.5615176374011, avg_score -187.422376472711, episode_time 0.77s, avg_episode_time 0.54s, avg_step_time 0.005815s, avg_learn_time 0.005346s, avg_steps_per_episode 108.00
episode 4, score -42.49161030909543, avg_score -151.1896849318071, episode_time 0.45s, avg_episode_time 0.51s, avg_step_time 0.006132s, avg_learn_time 0.005605s, avg_steps_per_episode 98.50
episode 5, score -146.79816778229667, avg_score -150.31138150190503, episode_time 0.64s, avg_episode_time 0.54s, avg_step_time 0.006427s, avg_learn_time 0.005870s, avg_steps_per_episode 97.20
episode 6, score -389.49917577841455, avg_score -1

action_0,▅▄▇▅████▅▅▅▅▆▅▅▅▂▁▂▇▅▁▆▆▄▃▇▅▅█▅▆▇▅▆▆▅▁▃▅
action_1,▅▆█▁▇▂▆▂▆▄▆▃▄▄▇▁▅▆▄▃▆▅▆▅▆▃▅▅▁▁▅▇█▅▄▃▅█▆▃
actor_loss,▁▂██▆▅▃▂▂▂▁▂▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
actor_predictions,▇█▆▅▄▄▅▆▄▄▅▂▁▃▄▄▅▃▄▃▄▅▅▄▅▅▅▆▆▅▃▄▅▅▅▄▅▆▆▆
avg_reward,▇▆▆▃▂▁▂▃▃▃▃▃▂▂▅▆▆▇▇▇███▇▇▇▆▅▆▅▄▄▄▄▄▄▅▅▅▅
best,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
critic_loss,▁▅█▁█▁▁▁▁▆▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
critic_predictions,█▇▁▁▃▄▆▇▇▇█▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███
episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode_reward,▇▆▄▁▄▃▇▆▄█▄▄▂▇▆▆▇█▇▆▇▃▇▇▆▆▇▇▄▄▅▅▅▅▆▆▆▆▆▆
episode_time,▁▁▁▂▁▁▁▁▁▂▁▂▂▁▂▂▄▃▂▂▂▄▃▂▁▆▂█▁▁▂▂▁▁▁▂▁▁▁▁


In [ ]:
td3.actor_model.device